In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

hero_data.csv
test1.csv
test9.csv
train1.csv
train9.csv



In [2]:
heroes = pd.read_csv("../input/hero_data.csv")
train9 = pd.read_csv("../input/train9.csv")
train1 = pd.read_csv("../input/train1.csv")
test9 = pd.read_csv("../input/test9.csv")
test1 = pd.read_csv("../input/test1.csv")

In [ ]:
train9['dataset'] = "train9"
train1['dataset'] = "train1"
test9['dataset'] = "test9"
test1['dataset'] = "test1"

In [ ]:
temp = heroes.iloc[:,:5]

In [ ]:
roles_count = pd.DataFrame(temp['roles'].str.split(':').apply(len))
roles_count.columns = ['roles_count']
roles_count.dtypes

In [ ]:
roles = pd.get_dummies(temp['roles'].str.split(':', expand=True).stack()).sum(level=0).add_prefix('roles_')

In [ ]:
heroes_roles = pd.concat([heroes,roles, roles_count ], axis=1)

In [ ]:
heroes_roles.columns

In [ ]:
drop_hero_columns=['roles']
heroes_roles.drop(drop_hero_columns,axis=1, inplace=True)


In [ ]:
heroes_roles.head()

In [ ]:
heroes_roles['custom_proj_move'] = heroes_roles['projectile_speed'] / heroes_roles['move_speed']
heroes_roles['custom_attach_turn'] = heroes_roles['attack_rate'] / heroes['turn_rate']
heroes_roles['custom_baseStrength_range'] = heroes_roles['base_strength'] / heroes['attack_range']
heroes_roles['custom_str_agi'] = heroes_roles['base_strength'] / heroes['base_agility']
heroes_roles['custom_str_int'] = heroes_roles['base_strength'] / heroes['base_intelligence']
heroes_roles['custom_agi_int'] = heroes_roles['base_agility'] / heroes['base_intelligence']

In [ ]:
one_hot_hero_primary_attr = pd.get_dummies(heroes_roles.primary_attr)
one_hot_hero_attack_type = pd.get_dummies(heroes_roles.attack_type)

In [ ]:
#removing categorical features 
heroes_roles.drop(['primary_attr','attack_type'],axis=1,inplace=True)

In [ ]:
heroes_roles = pd.concat([heroes_roles, one_hot_hero_attack_type, one_hot_hero_primary_attr], axis = 1)

In [ ]:
heroes_roles.head()

In [ ]:
#removing dulpicate columns 
_, i = np.unique(heroes_roles.columns, return_index=True)
heroes_roles=heroes_roles.iloc[:, i] 

In [ ]:
heroes_roles.head()

In [ ]:
heroes_roles.describe().T

In [ ]:
user_avg = train9[['user_id', 'kda_ratio']].groupby(['user_id']).agg('mean')
user_avg.reset_index(inplace=True)
user_avg.columns = ['user_id', 'kda_avg']

In [ ]:
user_avg.shape

In [ ]:
full_data_train = pd.concat([train9, train1])

In [ ]:
full_data_train.head()

In [ ]:
train_merge = pd.merge(full_data_train, heroes_roles, on="hero_id")
train_merge_avg = pd.merge(train_merge, user_avg, on="user_id")
train_merge = train_merge_avg

In [ ]:
#removing categorical features 
train_merge.drop(['id','num_wins'],axis=1,inplace=True)

In [ ]:
# train_merge.to_csv("output.csv")

In [ ]:
train = train_merge[train_merge['dataset'] == 'train9']
test = train_merge[train_merge['dataset'] == 'train1']

In [ ]:
#removing categorical features 
train.drop(['dataset'],axis=1,inplace=True)
test.drop(['dataset'],axis=1,inplace=True)

In [ ]:
x_train = train.drop('kda_ratio', axis=1)
y_train = train.kda_ratio

x_test = test.drop('kda_ratio', axis=1)
y_test = test.kda_ratio

In [ ]:
x_test.shape

### XGBoost Model

In [ ]:
#import lightgbm and xgboost 
import lightgbm as lgb 
import xgboost as xgb 

import numpy as np
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston

%matplotlib inline
import seaborn as sns
import pandas as pd

In [ ]:
#The data is stored in a DMatrix object 
#label is used to define our outcome variable
dtrain=xgb.DMatrix(x_train,label=y_train)
dtest=xgb.DMatrix(x_test)

In [ ]:
# Test Start

In [ ]:
xgb_model = xgb.XGBRegressor().fit(x_train, y_train)
predictions = xgb_model.predict(x_test)
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
print("Parameter optimization")
y = y_train
X = x_train
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [6],
                    'learning_rate': [0.05, 0.1, 0.3],
                    'subsample': [0.9, 1.0],
                    'colsample_bytree': [0.9, 1.0],
                    'n_estimators': [500]}, verbose=1)
clf.fit(X,y)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 200
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train, y_train)
predictions = xgb_model.predict(x_test)
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 1000
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train, y_train)
predictions = xgb_model.predict(x_test)
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
predictions

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 5000
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train, y_train)
predictions = xgb_model.predict(x_test)
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 1000,
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'subsample': 0.9,
    'eval_metric': 'rmse'
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train, y_train)
predictions = xgb_model.predict(x_test)
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
# Test End

In [ ]:
#setting parameters for xgboost
parameters={'max_depth':7, 'eta':1, 'silent':1,'objective':'reg:linear','eval_metric':'rmse','learning_rate':.05}
num_round=10

In [ ]:
xg=xgb.train(parameters,dtrain,num_round) 

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 16)
sns.set(font_scale = 2.5)
xgb.plot_importance(xgb_model, importance_type='gain', xlabel='Gain')

In [ ]:
ypred=xg.predict(dtest) 
ypred

In [ ]:
print(mean_squared_error(y_test, ypred))

### Predictions

In [ ]:
test9.shape

In [ ]:
test_user_avg = test9[['user_id', 'kda_ratio']].groupby(['user_id']).agg('mean')
test_user_avg.reset_index(inplace=True)
test_user_avg.columns = ['user_id', 'kda_avg']

full_data_pred = pd.concat([test9, test1])
pred_merge = pd.merge(full_data_pred, heroes_roles, on="hero_id")
pred_merge_avg = pd.merge(pred_merge, test_user_avg, on="user_id")
pred_merge = pred_merge_avg

#removing categorical features 
pred_merge.drop(['id','num_wins'],axis=1,inplace=True)
#pred_merge.drop(['id'],axis=1,inplace=True)

pred_train = pred_merge[pred_merge['dataset'] == 'test9']
pred_test = pred_merge[pred_merge['dataset'] == 'test1']

#removing categorical features 
pred_train.drop(['dataset'],axis=1,inplace=True)
pred_test.drop(['dataset'],axis=1,inplace=True)

In [ ]:
x_pred_train = pred_train.drop('kda_ratio', axis=1)
y_pred_train = pred_train.kda_ratio

x_pred_test = pred_test.drop('kda_ratio', axis=1)

#The data is stored in a DMatrix object 
#label is used to define our outcome variable
dtrain_pred=xgb.DMatrix(x_pred_train,label=y_pred_train)
dtest_pred=xgb.DMatrix(x_pred_test)

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 1000,
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'subsample': 0.9,
    'eval_metric': 'rmse'
}
xgb_model = xgb.XGBRegressor(**params).fit(x_pred_train, y_pred_train)
predictions = xgb_model.predict(x_pred_test)
#actuals = y_test
#print(mean_squared_error(actuals, predictions))

In [ ]:
predictions

In [ ]:
xgb_model = xgb.XGBRegressor().fit(x_pred_train, y_pred_train)
predictions = xgb_model.predict(x_pred_test)
#actuals = y_test
#print(mean_squared_error(actuals, predictions))

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 1000,
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'subsample': 0.9
}
xgb_model = xgb.XGBRegressor(**params).fit(x_pred_train, y_pred_train)
predictions = xgb_model.predict(x_pred_test)
predictions

In [ ]:
#setting parameters for xgboost
parameters={'max_depth':7, 'eta':1, 'silent':1,'objective':'reg:linear','eval_metric':'rmse','learning_rate':.05}
num_round=10

In [ ]:
xg_pred=xgb.train(parameters,dtrain_pred,num_round) 

In [ ]:
ypred_final=xg_pred.predict(dtest_pred) 
ypred_final

In [ ]:
submission = pd.DataFrame({ 'id': test1['id'],
                            'kda_ratio': ypred_final })
submission.to_csv("submission.csv", index=False)

### Light GBM

In [ ]:
#import lightgbm and xgboost 
import lightgbm as lgb 

In [ ]:
train_data_pred=lgb.Dataset(x_pred_train,label=y_pred_train)

In [ ]:
#setting parameters for lightgbm
param = {'num_leaves':150, 'objective':'regression','max_depth':7,'learning_rate':.05,'max_bin':200}
param['metric'] = ['rmse']

#training our model using light gbm
num_round=50
lgbm=lgb.train(param,train_data_pred,num_round)

In [ ]:
# ALL

In [ ]:
col_order = ['hero_id', 'num_games', 'user_id', 'Melee', 'Ranged', 'agi', 'agility_gain', 'attack_range', 'attack_rate', 'base_agility', 'base_armor', 'base_attack_max', 'base_attack_min', 'base_health', 'base_health_regen', 'base_intelligence', 'base_magic_resistance', 'base_mana', 'base_mana_regen', 'base_strength', 'int', 'intelligence_gain', 'move_speed', 'projectile_speed', 'roles_Carry', 'roles_Disabler', 'roles_Durable', 'roles_Escape', 'roles_Initiator', 'roles_Jungler', 'roles_Nuker', 'roles_Pusher', 'roles_Support', 'str', 'strength_gain', 'turn_rate']

x_train_all = pd.concat([x_train[col_order], x_pred_train[col_order]])
y_train_all = pd.concat([y_train, y_pred_train])

# x_test = test.drop('kda_ratio', axis=1)
# y_test = test.kda_ratio
#x_pred_test = pred_test.drop('kda_ratio', axis=1)

dtrain_all=xgb.DMatrix(x_train_all,label=y_train_all)
dtest=xgb.DMatrix(x_test)

dtrain_pred=xgb.DMatrix(x_pred_train,label=y_pred_train)
dtest_pred=xgb.DMatrix(x_pred_test)

In [ ]:
 x_train[col_order].columns.tolist() == x_pred_train[col_order].columns.tolist()
# x_train.columns.tolist()

#pd.DataFrame(x_train.columns.tolist(), x_pred_train.columns.tolist())

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 200
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train_all, y_train_all)

In [ ]:
predictions = xgb_model.predict(x_test[col_order])
actuals = y_test
print(mean_squared_error(actuals, predictions))

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 200
}
xgb_model = xgb.XGBRegressor(**params).fit(x_train_all, y_train_all)
predictions = xgb_model.predict(x_pred_test)
#actuals = y_test
#print(mean_squared_error(actuals, predictions))

In [ ]:
predictions